In [43]:
from google.colab import drive

# Google Driveをマウント
drive.mount('/content/drive')

# マウント完了後のメッセージ
print("Google Driveがマウントされました！")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Driveがマウントされました！


In [44]:
import pandas as pd

# CSVファイルを読み込む
file_path = '/content/drive/My Drive/signate/train.csv'
df = pd.read_csv(file_path, low_memory= False)

# 読み込んだデータを表示
df.head()


,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status
0,88194295,1800.0,3 years,14.49,C4,NaN,debt_consolidation,665.0,Individual,FullyPaid
1,5146039,1200.0,5 years,16.29,C4,2 years,debt_consolidation,700.0,Individual,ChargedOff
2,3095896,2000.0,5 years,21.98,E4,10 years,home_improvement,670.0,Individual,FullyPaid
3,88625044,1000.0,3 years,8.59,A5,4 years,debt_consolidation,710.0,Individual,FullyPaid
4,1178189,1500.0,3 years,13.99,C1,4 years,debt_consolidation,680.0,Individual,FullyPaid


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242156 entries, 0 to 242155
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 242156 non-null  int64  
 1   loan_amnt          242156 non-null  float64
 2   term               242156 non-null  object 
 3   interest_rate      242156 non-null  float64
 4   grade              242156 non-null  object 
 5   employment_length  228971 non-null  object 
 6   purpose            242156 non-null  object 
 7   credit_score       242156 non-null  float64
 8   application_type   242156 non-null  object 
 9   loan_status        242156 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 18.5+ MB


In [26]:
# 列のユニークな値
print("Unique values in 'employment_length':", df['employment_length'].unique())

# 列のユニークな値の総数
unique_count = df['employment_length'].nunique(dropna=False)

print(f"Total unique values in 'employment_length': {unique_count}")

Unique values in 'employment_length': [nan '2 years' '10 years' '4 years' '3 years' '7 years' '5 years'
 '8 years' '1 year' '6 years' '0 years' '9 years']
Total unique values in 'employment_length': 12


In [45]:
import re
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle

# id列を削除
df = df.drop(columns=['id'])

# 'employment_length' の処理関数
def process_employment_length(value):
    if pd.isna(value):  # NaN の場合
        return np.nan
    # 正規表現で 'years' と 'year' を削除し、'< 1' を '0' に置き換え
    value = re.sub(r' years| year', '', value)
    value = re.sub(r'< 1', '0', value)
    return float(value.strip())  # 数値型に変換

# employment_length 列の処理
df['employment_length'] = df['employment_length'].apply(process_employment_length)

# 'term' の処理
df['term'] = df['term'].apply(
    lambda x: float(re.sub(r' years', '', x).strip()) if isinstance(x, str) else np.nan
)

# interest_rate の正規化
df['interest_rate'] = df['interest_rate'] / 100  # パーセントを小数に変換

# grade のラベルエンコーディング
grade_encoder = LabelEncoder()
df['grade_encoded'] = grade_encoder.fit_transform(df['grade'])


# Google Drive のパス
drive_path = '/content/drive/My Drive/grade_encoder.pkl'

# エンコーダーの保存
with open(drive_path, 'wb') as file:
    pickle.dump(grade_encoder, file)

# 保存した場所を表示
print(f"Encoder saved at: {drive_path}")

# 月利と返済回数の準備
df['monthly_interest_rate'] = df['interest_rate'] / 12  # 月利に変換
df['term_months'] = df['term'] * 12  # 返済回数（月数）

# 毎月の返済額を計算する関数
def calculate_monthly_payment(loan_amnt, monthly_rate, term_months):
    if monthly_rate > 0:  # 月利が0でない場合
        return loan_amnt * (monthly_rate * (1 + monthly_rate)**term_months) / ((1 + monthly_rate)**term_months - 1)
    else:  # 月利が0の場合（単純に元金を均等割）
        return loan_amnt / term_months

# 毎月の返済額を計算
df['monthly_payment'] = df.apply(lambda row: calculate_monthly_payment(
    row['loan_amnt'], row['monthly_interest_rate'], row['term_months']
), axis=1)

# 総返済額（毎月の返済額 × 返済回数）
df['total_payment'] = df['monthly_payment'] * df['term_months']

# 負担指数の計算
df['burden_index'] = df['total_payment'] / df['credit_score']

# FullyPaidを0、ChargedOffを1に変換
df['loan_status'] = df['loan_status'].map({'FullyPaid': 0, 'ChargedOff': 1}).astype(int)


# 結果を確認
df.head()


Encoder saved at: /content/drive/My Drive/grade_encoder.pkl


,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status,grade_encoded,monthly_interest_rate,term_months,monthly_payment,total_payment,burden_index
0,1800.0,3.0,0.1449,C4,NaN,debt_consolidation,665.0,Individual,0,13,0.012075,36.0,61.948981,2230.163322,3.353629
1,1200.0,5.0,0.1629,C4,2.0,debt_consolidation,700.0,Individual,1,13,0.013575,60.0,29.366894,1762.013668,2.517162
2,2000.0,5.0,0.2198,E4,10.0,home_improvement,670.0,Individual,0,23,0.018317,60.0,55.215084,3312.905027,4.944634
3,1000.0,3.0,0.0859,A5,4.0,debt_consolidation,710.0,Individual,0,4,0.007158,36.0,31.609257,1137.933255,1.602723
4,1500.0,3.0,0.1399,C1,4.0,debt_consolidation,680.0,Individual,0,10,0.011658,36.0,51.259160,1845.329745,2.713720


In [28]:
# 結果を確認
print(df['loan_status'].value_counts())

loan_status
0    193815
1     48341
Name: count, dtype: int64


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242156 entries, 0 to 242155
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   loan_amnt              242156 non-null  float64
 1   term                   242156 non-null  float64
 2   interest_rate          242156 non-null  float64
 3   grade                  242156 non-null  object 
 4   employment_length      228971 non-null  float64
 5   purpose                242156 non-null  object 
 6   credit_score           242156 non-null  float64
 7   application_type       242156 non-null  object 
 8   loan_status            242156 non-null  int64  
 9   grade_encoded          242156 non-null  int64  
 10  monthly_interest_rate  242156 non-null  float64
 11  term_months            242156 non-null  float64
 12  monthly_payment        242156 non-null  float64
 13  total_payment          242156 non-null  float64
 14  burden_index           242156 non-nu

# 新しい特徴量を作成する

In [46]:
import numpy as np

# total_employment_length の計算
df["total_employment_length"] = np.where(
    df["employment_length"].isna(),  # employment_length が NaN かをチェック
    0 + df["term"],                 # NaN の場合
    df["employment_length"] + df["term"]  # NaN でない場合
)


# 結果の確認
df.head()


,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status,grade_encoded,monthly_interest_rate,term_months,monthly_payment,total_payment,burden_index,total_employment_length
0,1800.0,3.0,0.1449,C4,NaN,debt_consolidation,665.0,Individual,0,13,0.012075,36.0,61.948981,2230.163322,3.353629,3.0
1,1200.0,5.0,0.1629,C4,2.0,debt_consolidation,700.0,Individual,1,13,0.013575,60.0,29.366894,1762.013668,2.517162,7.0
2,2000.0,5.0,0.2198,E4,10.0,home_improvement,670.0,Individual,0,23,0.018317,60.0,55.215084,3312.905027,4.944634,15.0
3,1000.0,3.0,0.0859,A5,4.0,debt_consolidation,710.0,Individual,0,4,0.007158,36.0,31.609257,1137.933255,1.602723,7.0
4,1500.0,3.0,0.1399,C1,4.0,debt_consolidation,680.0,Individual,0,10,0.011658,36.0,51.259160,1845.329745,2.713720,7.0


In [47]:
import os

# 保存先のディレクトリを指定
output_dir = "/content/drive/My Drive/signate"
os.makedirs(output_dir, exist_ok=True)  # ディレクトリが存在しない場合は作成

# 保存ファイル名を指定
output_file = os.path.join(output_dir, "train_1221_5.csv")

# データセットを保存
df.to_csv(output_file, index=False)

print(f"データセットが保存されました: {output_file}")

データセットが保存されました: /content/drive/My Drive/signate/train_1221_5.csv


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242156 entries, 0 to 242155
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   loan_amnt                242156 non-null  float64
 1   term                     242156 non-null  float64
 2   interest_rate            242156 non-null  float64
 3   grade                    242156 non-null  object 
 4   employment_length        228971 non-null  float64
 5   purpose                  242156 non-null  object 
 6   credit_score             242156 non-null  float64
 7   application_type         242156 non-null  object 
 8   loan_status              242156 non-null  int64  
 9   grade_encoded            242156 non-null  int64  
 10  monthly_interest_rate    242156 non-null  float64
 11  term_months              242156 non-null  float64
 12  monthly_payment          242156 non-null  float64
 13  total_payment            242156 non-null  float64
 14  burd